In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/20240427_161502.jpg
/kaggle/input/dataset/IMG_5699.jpeg
/kaggle/input/dataset/20240427_161434.jpg
/kaggle/input/dataset/20240427_161527.jpg
/kaggle/input/dataset/IMG-3736.jpg


In [2]:
import os
import shutil

# Make internal folder
os.makedirs("/kaggle/working/dataset/all_images", exist_ok=True)

# Copy all images from your input dataset to the new internal folder
for file in os.listdir("/kaggle/input/dataset"):
    src = os.path.join("/kaggle/input/dataset", file)
    dst = os.path.join("/kaggle/working/dataset/all_images", file)
    if os.path.isfile(src):  # only copy files
        shutil.copy(src, dst)

print("Internal folder created with images.")


Internal folder created with images.


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms 
from torchvision.utils import save_image
import os
from torchvision.datasets import ImageFolder


latent_dim = 100
img_channels = 3
img_size = 32
batch_size = 1
lr = 0.0002
beta1 = 0.5
epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("dcgan_outputs", exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

dataset = ImageFolder(root='/kaggle/working/dataset', transform=transform)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model=nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0), #in_chan,out_chan, kernel, stride, padd
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.ConvTranspose2d(512, 256, 4, 2,1), #in_chan,out_chan, kernel, stride, padd
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),       # (B,128,16,16)
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, 4, 2, 1),# (B,3,32,32)
            nn.Tanh()
        )
    def forward(self,z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model= nn.Sequential(
            nn.Conv2d(img_channels, 128, 4,2,1), #[B,128, 16, 16]
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128,256,4,2,1),   #8*8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),  #4*4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512,1,4,1,0),
            nn.Sigmoid()


        )
    def forward(self, img):
        out=self.model(img)
        return out.view(-1,1)


model_Gen= Generator().to(device)
model_Gen

model_Dis=Discriminator().to(device)

model_Dis

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (9): Sigmoid()
  )
)

In [5]:
loss_fn=nn.BCELoss()

optimizer_G=torch.optim.Adam(params=model_Gen.parameters(),lr=lr, betas=(beta1, 0.999))
optimizer_D=torch.optim.Adam(params=model_Dis.parameters(),lr=lr, betas=(beta1, 0.999))


fixed_noise=torch.randn(64, latent_dim, 1, 1, device=device)

In [6]:
# epochs=1
# k=0
# for i, (imgs, _) in enumerate(train_loader):
#     if i>1:
#         break
#     imgs=imgs.to(device)
#     batch=imgs.size(0)
#     real_label=torch.full((batch, 1), 0.9, device=device)
#     fake_label=torch.zeros(batch, 1).to(device)
#     # real_out=model_Dis()
#     print(imgs.shape)
#     #trainging discriminator
#     optimizer_D.zero_grad()
#     z=torch.randn(batch, latent_dim, 1, 1, device=device)
#     real_out=model_Dis(imgs)
#     real_loss=loss_fn(real_out, real_label)

#     fake_imgs=model_Gen(z)
#     fake_out=model_Dis(fake_imgs.detach())
#     fake_loss=loss_fn(fake_out, fake_label)

#     total_loss=real_loss+fake_loss
#     print(total_loss.item())
#     total_loss.backward()
#     optimizer_D.step()
#     #Generator training
#     optimizer_G.zero_grad()


#     gen_img=model_Gen(z)
#     dis_out_val=model_Dis(gen_img)

#     gen_loss=loss_fn(dis_out_val,real_label)
#     print(gen_loss.item())
#     gen_loss.backward()
#     optimizer_G.step()


#     if i%1==0:
#         print(f"i->{i} D[{total_loss.item()}] G[{gen_loss.item()}]")

#     with torch.no_grad():
#         fake_sample=model_Gen(fixed_noise).detach().cpu()
#         save_image(fake_sample, f"dcgan_outputs/fake_{epochs:03d}_{k:04d}.png", normalize=True)



# torch.save(model_Gen.state_dict(),"dcgan_generate.pth")
# torch.save(model_Dis.state_dict(),"dcgan_discriminator.pth")

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        return self.model(z)

# ========== Discriminator ==========
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(img_channels, 128, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0),
            nn.Sigmoid()
        )

    def forward(self, img):
        out = self.model(img)
        return out.view(-1, 1)

# ========== Model Initialization ==========
model_G = Generator().to(device)
model_D = Discriminator().to(device)

# ========== Loss & Optimizers ==========
loss_fn = nn.BCELoss()
optimizer_G = torch.optim.Adam(model_G.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = torch.optim.Adam(model_D.parameters(), lr=lr, betas=(beta1, 0.999))

# ========== Fixed Noise ==========
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

# ========== Training Loop ==========
k = 0
for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(train_loader):
        current_batch = real_imgs.size(0)
        real_imgs = real_imgs.to(device)

        # Real and fake labels
        real_label = torch.full((current_batch, 1), 0.9, device=device)
        fake_label = torch.zeros(current_batch, 1, device=device)

        # ======== Train Discriminator ========
        optimizer_D.zero_grad()

        real_out = model_D(real_imgs)
        real_loss = loss_fn(real_out, real_label)

        noise = torch.randn(current_batch, latent_dim, 1, 1, device=device)
        fake_imgs = model_G(noise)
        fake_out = model_D(fake_imgs.detach())
        fake_loss = loss_fn(fake_out, fake_label)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # ======== Train Generator ========
        optimizer_G.zero_grad()
        output = model_D(fake_imgs)
        g_loss = loss_fn(output, real_label)
        g_loss.backward()
        optimizer_G.step()

        # ======== Logging (optional but helpful) ========
        if i % 1 == 0:
            print(f"[Epoch {epoch+1}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

        # ======== Save Sample Images ========
        if i % 1 == 0:
            with torch.no_grad():
                fake_sample = model_G(fixed_noise).detach().cpu()
                save_image(fake_sample, f"dcgan_outputs/fake_{epoch:03d}_{k:04d}.png", normalize=True)
            k += 1

# ========== Save Final Model ==========
torch.save(model_G.state_dict(), "dcgan_generator.pth")
torch.save(model_D.state_dict(), "dcgan_discriminator.pth")

[Epoch 1/100] [Batch 0/5] [D loss: 1.2498] [G loss: 7.6156]
[Epoch 1/100] [Batch 1/5] [D loss: 0.8008] [G loss: 8.2821]
[Epoch 1/100] [Batch 2/5] [D loss: 1.4050] [G loss: 5.7704]
[Epoch 1/100] [Batch 3/5] [D loss: 2.1668] [G loss: 3.7015]
[Epoch 1/100] [Batch 4/5] [D loss: 1.0039] [G loss: 6.6485]
[Epoch 2/100] [Batch 0/5] [D loss: 0.5430] [G loss: 5.3008]
[Epoch 2/100] [Batch 1/5] [D loss: 1.1481] [G loss: 4.9499]
[Epoch 2/100] [Batch 2/5] [D loss: 0.7286] [G loss: 6.6641]
[Epoch 2/100] [Batch 3/5] [D loss: 1.0079] [G loss: 5.0679]
[Epoch 2/100] [Batch 4/5] [D loss: 0.9108] [G loss: 5.0171]
[Epoch 3/100] [Batch 0/5] [D loss: 0.9041] [G loss: 6.9457]
[Epoch 3/100] [Batch 1/5] [D loss: 0.5642] [G loss: 5.2304]
[Epoch 3/100] [Batch 2/5] [D loss: 0.3749] [G loss: 4.7262]
[Epoch 3/100] [Batch 3/5] [D loss: 1.9815] [G loss: 4.3939]
[Epoch 3/100] [Batch 4/5] [D loss: 0.9562] [G loss: 6.1994]
[Epoch 4/100] [Batch 0/5] [D loss: 0.4996] [G loss: 5.4762]
[Epoch 4/100] [Batch 1/5] [D loss: 0.600